In [ ]:
import sys
#kaggle_env = False

#if not kaggle_env:
if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#if not kaggle_env:
if 'google.colab' in sys.modules:
  !pip install kaggle --upgrade

  import os
  import json
  f = open("./drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
  json_data = json.load(f)
  os.environ["KAGGLE_USERNAME"] = json_data["username"]
  os.environ["KAGGLE_KEY"] = json_data["key"]

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [ ]:
#if not kaggle_env:
if 'google.colab' in sys.modules:
  # Change current directory
  %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative


In [ ]:
#if 'google.colab' in sys.modules:
  # Download Kaggle datasets related to this notebook
  ##!kaggle datasets list -s shahules/ner-coleridge-initiative
  #!mkdir input/ner-coleridge-initiative
  #!kaggle datasets download shahules/ner-coleridge-initiative -p input/ner-coleridge-initiative
  #!unzip input/ner-coleridge-initiative/ner-coleridge-initiative.zip -d input/ner-coleridge-initiative
  #!rm input/ner-coleridge-initiative/ner-coleridge-initiative.zip

In [ ]:
#if not kaggle_env:
if 'google.colab' in sys.modules:
  %cd notebooks

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks


In [ ]:
#!pip install NERDA
!git clone https://github.com/riow1983/NERDA.git
%cd NERDA
!pip install -e .
%cd ../
!pwd

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial

import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

#if kaggle_env:
if 'kaggle_web_client' in sys.modules:
    import nltk
    import spacy
    nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
    nlp.max_length = 4000000
    from nltk.probability import FreqDist
    from wordcloud import WordCloud, STOPWORDS

from tqdm.autonotebook import tqdm
import string

%matplotlib inline

#if kaggle_env:
if 'kaggle_web_client' in sys.modules:
    os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')
#else:
elif 'google.colab' in sys.modules:
    os.listdir('../input/coleridgeinitiative-show-us-the-data/')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

In [ ]:
import gc

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
#import NERDA

In [ ]:
#from NERDA.models import NERDA

In [ ]:
# reading csv files and train & test file paths
#train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
#sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
#train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
#test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
#if kaggle_env:
"""
if 'kaggle_web_client' in sys.modules:
    dirname = "/kaggle/input/coleridgeinitiative-show-us-the-data"
#else:
elif 'google.colab' in sys.modules:
    dirname = "../input/coleridgeinitiative-show-us-the-data"
"""
#df = pd.read_csv(os.path.join(dirname, "train.csv"))

'\nif \'kaggle_web_client\' in sys.modules:\n    dirname = "/kaggle/input/coleridgeinitiative-show-us-the-data"\n#else:\nelif \'google.colab\' in sys.modules:\n    dirname = "../input/coleridgeinitiative-show-us-the-data"\n'

In [ ]:
# Credit to: https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230341
#df = pd.read_pickle("../input/ner-coleridge-initiative/show_us_the_data_train_ner.pkl")
df = pd.read_pickle("../input/ner-coleridge-initiative/show_us_the_data_train_ner.pkl")

In [ ]:
df

,token,sentence#,sentence,entity
0,This,sentence#0,sentence0,o
1,study,sentence#0,sentence0,o
2,used,sentence#0,sentence0,o
3,data,sentence#0,sentence0,o
4,from,sentence#0,sentence0,o
...,...,...,...,...
53139372,to,sentence#15,sentence9999,o
53139373,many,sentence#15,sentence9999,o
53139374,other,sentence#15,sentence9999,o
53139375,biological,sentence#15,sentence9999,o


In [ ]:
df_train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")

def get_text(filename, test=False):
    if test:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(filename))
    else:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(filename))
    text = " ".join(list(df['text']))
    return text

df_train['text'] = df_train['Id'].apply(get_text)

POS tagging using spaCy  
refenrence: https://spacy.io/usage/linguistic-features

In [ ]:
#tmp = df.groupby(["sentence", "sentence#"]).apply(lambda r: nlp(" ".join(r["token"].tolist()))).reset_index()
#tmp = df.groupby(["sentence", "sentence#"]).apply(lambda r: " ".join(r["token"].tolist())).reset_index()

In [ ]:
pos = []
for doc in nlp.pipe(df_train['text'].values, batch_size=50, n_process=4):
    if doc.is_parsed:
        pos.append([n.pos_ for n in doc])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        pos.append(None)

In [ ]:
df_train["pos"] = pos
df_train

In [ ]:
res = []
for doc in tmp["token"].tolist():
    for token in doc:
        res.append(token.pos_)

In [ ]:
df["pos"] = res

In [ ]:
df["token"].nunique()

857922

In [ ]:
df["sentence#"].value_counts(dropna=False)

sentence#0      2899042
sentence#1      2892327
sentence#2      2876816
sentence#3      2852276
sentence#4      2827468
                 ...   
sentence#209        290
sentence#201        290
sentence#236        290
sentence#223        290
sentence#242         72
Name: sentence#, Length: 243, dtype: int64

In [ ]:
df["sentence#"].unique()

array(['sentence#0', 'sentence#1', 'sentence#2', 'sentence#3',
       'sentence#4', 'sentence#5', 'sentence#6', 'sentence#7',
       'sentence#8', 'sentence#9', 'sentence#10', 'sentence#11',
       'sentence#12', 'sentence#13', 'sentence#14', 'sentence#15',
       'sentence#16', 'sentence#17', 'sentence#18', 'sentence#19',
       'sentence#20', 'sentence#21', 'sentence#22', 'sentence#23',
       'sentence#24', 'sentence#25', 'sentence#26', 'sentence#27',
       'sentence#28', 'sentence#29', 'sentence#30', 'sentence#31',
       'sentence#32', 'sentence#33', 'sentence#34', 'sentence#35',
       'sentence#36', 'sentence#37', 'sentence#38', 'sentence#39',
       'sentence#40', 'sentence#41', 'sentence#42', 'sentence#43',
       'sentence#44', 'sentence#45', 'sentence#46', 'sentence#47',
       'sentence#48', 'sentence#49', 'sentence#50', 'sentence#51',
       'sentence#52', 'sentence#53', 'sentence#54', 'sentence#55',
       'sentence#56', 'sentence#57', 'sentence#58', 'sentence#59',
     

In [ ]:
df["sentence"].value_counts(dropna=False)

sentence6030    70252
sentence2770    57077
sentence950     57077
sentence7045    39753
sentence3418    38432
                ...  
sentence3724      213
sentence8659      213
sentence3793       63
sentence8653       63
sentence8652       57
Name: sentence, Length: 10000, dtype: int64

In [ ]:
df["sentence"].unique()

array(['sentence0', 'sentence1', 'sentence2', ..., 'sentence9997',
       'sentence9998', 'sentence9999'], dtype=object)

In [ ]:
pd.options.display.max_rows = 60 #9999

In [ ]:
df[df["sentence"]=="sentence9999"]

,token,sentence#,sentence,entity
53134924,Motivation:,sentence#0,sentence9999,o
53134925,The,sentence#0,sentence9999,o
53134926,identification,sentence#0,sentence9999,o
53134927,of,sentence#0,sentence9999,o
53134928,quantitative,sentence#0,sentence9999,o
...,...,...,...,...
53139372,to,sentence#15,sentence9999,o
53139373,many,sentence#15,sentence9999,o
53139374,other,sentence#15,sentence9999,o
53139375,biological,sentence#15,sentence9999,o


In [ ]:
df[df["sentence"]=="sentence0"]

,token,sentence#,sentence,entity
0,This,sentence#0,sentence0,o
1,study,sentence#0,sentence0,o
2,used,sentence#0,sentence0,o
3,data,sentence#0,sentence0,o
4,from,sentence#0,sentence0,o
...,...,...,...,...
1759,(version,sentence#6,sentence0,o
1760,2.1),sentence#6,sentence0,o
1761,for,sentence#6,sentence0,o
1762,additional,sentence#6,sentence0,o


#### Data Preparation for NERD fine-tuning
- data structure:  
    {'sentences': [[], [], ..., []],  
     'tags': [[], [], ..., []]}

In [ ]:
#tmp = df.groupby("sentence")["entity"].count()

In [ ]:
# CV (GroupKfold)
"""example
import numpy as np
from sklearn.model_selection import GroupKFold
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

print(group_kfold)

for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)
"""

X = df.index.values
y = df["entity"].values
groups = df["sentence"].values

group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(X, y, groups)

print(group_kfold)

res = {}
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, groups)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #res[i] = [X_train, y_train, X_test, y_test]
    res[i] = [X_train, X_test]

GroupKFold(n_splits=5)


In [ ]:
# ToDo: save res as a pickle file
#pickle.dump(res, open("nb003_res_01.pkl", "wb"))
#del res
#gc.collect()


#!kaggle datasets create -p nb003_res_01.npy

In [ ]:
#res = pickle.load(open("nb003_res_01.pkl", "rb"))

In [ ]:
# train / dev split
#train, dev = train_test_split(df, test_size=0.3, random_state=42, shuffle=True, stratify=df["sentence"])

In [ ]:
dname = "nb003-annotation-data"
!mkdir {dname}

for i in res.keys():
    X_train = res[i][0]
    X_test = res[i][1]

    pickle.dump(df[df.index.isin(X_train)], open(f"nb003_cv{i+1}_train.pkl", "wb"))
    pickle.dump(df[df.index.isin(X_test)], open(f"nb003_cv{i+1}_test.pkl", "wb"))
    
    !mv nb003_cv{i+1}_train.pkl {dname}/nb003_cv{i+1}_train.pkl
    !mv nb003_cv{i+1}_test.pkl {dname}/nb003_cv{i+1}_test.pkl


!kaggle datasets init -p {dname}
# referene: https://kaeru-nantoka.hatenablog.com/entry/2020/01/17/015551

with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["id"] = f"riow1983/{dname}"
data["title"] = dname

with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
    json.dump(data, jsonFile)

!kaggle datasets create -p {dname}

del res, df, X_train, X_test, y_train, y_test
gc.collect()

Data package template written to: nb003-annotation-data/dataset-metadata.json
Starting upload for file nb003_cv1_train.pkl
100% 1.51G/1.51G [00:57<00:00, 28.3MB/s]
Upload successful: nb003_cv1_train.pkl (2GB)
Starting upload for file nb003_cv1_test.pkl
100% 387M/387M [00:16<00:00, 24.5MB/s]
Upload successful: nb003_cv1_test.pkl (387MB)
Starting upload for file nb003_cv2_train.pkl
100% 1.51G/1.51G [00:53<00:00, 30.5MB/s]
Upload successful: nb003_cv2_train.pkl (2GB)
Starting upload for file nb003_cv2_test.pkl
100% 387M/387M [00:13<00:00, 29.6MB/s]
Upload successful: nb003_cv2_test.pkl (387MB)
Starting upload for file nb003_cv3_train.pkl
100% 1.51G/1.51G [00:52<00:00, 31.1MB/s]
Upload successful: nb003_cv3_train.pkl (2GB)
Starting upload for file nb003_cv3_test.pkl
100% 387M/387M [00:15<00:00, 26.3MB/s]
Upload successful: nb003_cv3_test.pkl (387MB)
Starting upload for file nb003_cv4_train.pkl
100% 1.51G/1.51G [00:48<00:00, 33.3MB/s]
Upload successful: nb003_cv4_train.pkl (2GB)
Starting up

36

***
# NERDA training

Read CV datasets

In [ ]:
dname = "nb003-annotation-data"

In [ ]:
cv = 1
if 'google.colab' in sys.modules:
    train = pd.read_pickle(dname+f"/nb003_cv{cv}_train.pkl")
    dev = pd.read_pickle(dname+f"/nb003_cv{cv}_test.pkl")

In [ ]:
"""
data structure:
{'sentences': [[], [], ..., []],
'tags': [[], [], ..., []]}
"""

In [ ]:
#pd.options.display.max_rows = 999
#train[(train["sentence"]=="sentence0")&(train["sentence#"]=="sentence#0")].shape

(290, 4)

In [ ]:
#from NERDA.datasets import get_conll_data, download_conll_data 

In [ ]:
#download_conll_data()
#tmp = get_conll_data('train')
#validation = get_conll_data('valid')

In [ ]:
#for i in train.index:
#    print(i)

In [ ]:
sentences = train.groupby(["sentence", "sentence#"]).apply(lambda r: r["token"].tolist())
tags = train.groupby(["sentence", "sentence#"]).apply(lambda r: r["entity"].tolist())
train = {"sentences":sentences.tolist(), "tags":tags.tolist()}
del sentences, tags
gc.collect()

0

In [ ]:
sentences = dev.groupby(["sentence", "sentence#"]).apply(lambda r: r["token"].tolist())
tags = dev.groupby(["sentence", "sentence#"]).apply(lambda r: r["entity"].tolist())
dev = {"sentences":sentences.tolist(), "tags":tags.tolist()}
del sentences, tags
gc.collect()

0

In [ ]:
tag_scheme = ['o-dataset']
tag_outside = 'o'
# reference:
# [1] https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230341
# [2] https://towardsdatascience.com/easy-fine-tuning-of-transformers-for-named-entity-recognition-d72f2b5340e3

training_hyperparameters = {
    'epochs' : 4,
    'warmup_steps' : 500,    
    'train_batch_size': 5,        
    'learning_rate': 0.0001
}

dropout = 0.1

transformer = 'bert-base-multilingual-uncased'
max_len = 512

In [ ]:
help(NERDA.models)

AttributeError: ignored

In [ ]:
model = NERDA(
    dataset_training = train,
    dataset_validation = dev,
    tag_scheme = tag_scheme, 
    tag_outside = tag_outside,
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters,
    max_len = max_len)

TypeError: ignored

In [ ]:
model.train()



  0%|          | 0/30116 [00:00<?, ?it/s]


 Epoch 1 / 4




  0%|          | 1/30116 [00:00<4:24:03,  1.90it/s]

  0%|          | 2/30116 [00:00<3:33:11,  2.35it/s]

  0%|          | 3/30116 [00:00<2:57:07,  2.83it/s]

  0%|          | 4/30116 [00:01<2:32:22,  3.29it/s]

  0%|          | 5/30116 [00:01<2:14:46,  3.72it/s]

  0%|          | 6/30116 [00:01<2:02:21,  4.10it/s]

  0%|          | 7/30116 [00:01<1:53:47,  4.41it/s]

  0%|          | 8/30116 [00:01<1:47:16,  4.68it/s]

  0%|          | 9/30116 [00:02<1:42:57,  4.87it/s]

  0%|          | 10/30116 [00:02<1:40:28,  4.99it/s]

  0%|          | 11/30116 [00:02<1:39:05,  5.06it/s]

  0%|          | 12/30116 [00:02<1:37:58,  5.12it/s]

  0%|          | 13/30116 [00:02<1:36:45,  5.19it/s]

  0%|          | 14/30116 [00:02<1:35:30,  5.25it/s]

  0%|          | 15/30116 [00:03<1:34:36,  5.30it/s]

  0%|          | 16/30116 [00:03<1:33:55,  5.34it/s]

  0%|          | 17/30116 [00:03<1:33:51,  5.34it/s]

  0%|          | 18/30116 [00:03<1:34:10,  5.33it/s]/usr/local/lib/python3.7/dist-package

RuntimeError: ignored